In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.models import load_model
from tensorflow import keras
from pathlib import Path

from model import CNN_m123 as CNN
from utils import *

import os, sys


os.environ["CUDA_VISIBLE_DEVICES"] = "4,5,6,7"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"
print(tf.__version__)

2.1.0


# Train the model

In [3]:
def get_tf_model(n_classes):
    inputs = tf.keras.Input(shape=(16,100,1))
    x = layers.Conv2D(5, (5,5), padding='same')(inputs)
    x = layers.MaxPool2D((4,4))(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(10,(5,5), padding='same')(x)
    x = layers.MaxPool2D((2,2))(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(n_classes,(3,3), padding='same')(x)
    outputs = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Softmax()(outputs)
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="CNN_m123")
    return model

dir_data = Path('./data/datasets_fin/')


train_images = np.load(dir_data/'train_images_imu_mix.npy')
train_labels = np.load(dir_data/'train_labels_imu_mix.npy')
train_images = train_images[..., np.newaxis]

#train_data = np.load('./har_cnn/real-data/real_images_100_100_6_0826_M1+M2+M3.npy')
##train_data = np.load('./har_cnn/real-data/real_labels_100_100_6_0826_M1+M2+M3.npy')
#train_data = train_data.reshape(-1,1,16,100)

test_images = np.load(dir_data/'test_images_imu_mix.npy')
test_labels = np.load(dir_data/'test_labels_imu_mix.npy')
test_images = test_images[..., np.newaxis]

    
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = train_dataset.shuffle(buffer_size=10000).batch(128)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset = test_dataset.shuffle(buffer_size=10000).batch(128)

#compile and train

model = get_tf_model(n_classes=6)
model.summary()
def MultiStepLRSchedule(epoch, lr):
    milestones = [100]
    gamma = 5e-4
    print(lr)
    for i in range(1,len(milestones)+1):
        if(epoch == milestones[-i]):
            return lr*gamma
    return lr

callbacks = [tf.keras.callbacks.LearningRateScheduler(MultiStepLRSchedule)]
scheduler = tf.keras.optimizers.schedules.LearningRateSchedule
#opt = tf.keras.optimizers.SGD(learning_rate = 1e-5, momentum=0.9)
opt = tf.keras.optimizers.Adam(learning_rate = 1e-3)
model.compile(optimizer=opt , loss=keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])
hist = model.fit(train_dataset,
                 callbacks=callbacks,
                 validation_data = test_dataset,
                 epochs=200
                )

model.save('./models/harcnn.h5')

Model: "CNN_m123"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 16, 100, 1)]      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 100, 5)        130       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 25, 5)          0         
_________________________________________________________________
re_lu_2 (ReLU)               (None, 4, 25, 5)          0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 4, 25, 10)         1260      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 2, 12, 10)         0         
_________________________________________________________________
re_lu_3 (ReLU)               (None, 2, 12, 10)         0  

In [4]:
model.evaluate(test_dataset)

19/19 [==============================] - 0s 3ms/step - loss: 0.0535 - accuracy: 0.9833


[0.05351379405903189, 0.98333335]

# 32-bit Float model export

In [5]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model_fp = converter.convert()
with tf.io.gfile.GFile('./models/model_32fp.tflite', 'wb') as f:
    f.write(tflite_model_fp)

# 8-bit integer quantization

In [13]:
representative_dataset = tf.data.Dataset.from_tensor_slices(train_images.astype('float32'))
representative_dataset = representative_dataset.shuffle(buffer_size=10000).batch(1)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
def representative_dataset_gen():
    for i, samples in enumerate(representative_dataset.take(1)):
        yield [samples]
        
converter.representative_dataset = representative_dataset_gen
#converter.target_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
#converter.inference_input_type = tf.uint8
#converter.inference_output_type = tf.uint8
tflite_model_int = converter.convert()
with tf.io.gfile.GFile('./models/model_quant_8b.tflite', 'wb') as f:
    f.write(tflite_model_int)

None


In [63]:
# .npz file for validation
stm_valid_inputs = [];
stm_valid_labels = [];
cnt = np.zeros(6)
for j in range(len(test_labels)):
    one_shot_label = np.zeros(6)
    one_shot_label[test_labels[j]] = 1
    stm_valid_inputs.append(test_images[j].reshape(16,100,1))
    stm_valid_labels.append(one_shot_label)
    cnt[test_labels[j]] +=1


np.savez(f'stm_valid_inputs.npz', x_test=np.array(stm_valid_inputs), y_test=np.array(stm_valid_labels))
print(cnt)

[[2.2795305e-14 2.2497636e-04 1.6951935e-13 4.6011915e-27 9.9977499e-01
  0.0000000e+00]]
[[4]]
[400. 400. 400. 400. 400. 400.]
